In [1]:
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizerFast
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments
from transformers import BertConfig, BertForMaskedLM
from transformers import Trainer
import pandas as pd

In [2]:
data = pd.read_csv('dataset.csv')
data.head()

,text,label,link
0,В 2003-ем году под руководством малоизвестного...,0,dataset/neg/1000083-0.txt
1,"Грустно и печально. Грустно от того, что довол...",0,dataset/neg/1000083-1.txt
2,Давным-давно Кира Найтли ворвалась на экран от...,0,dataset/neg/1000125-3.txt
3,"Я, в общем, ничего против уравновешенного феми...",0,dataset/neg/1000125-4.txt
4,"Измена один из сюжетов, который всегда будет ...",0,dataset/neg/1000125-6.txt


In [3]:
# тренировка токенизатора на корпусе
bert_tokenizer = BertWordPieceTokenizer()
bert_tokenizer.train('corpus.txt')

In [4]:
len(bert_tokenizer.get_vocab())

30000

In [5]:
# проверка токенизатора
tokenized_sentence = bert_tokenizer.encode('Приветствую вас граждане !')
tokenized_sentence.tokens

['привет', '##ствую', 'вас', 'граждане', '!']

In [6]:
# сохранениние токенизатора
bert_tokenizer.save_model('tokenizer')
tokenizer = BertWordPieceTokenizer.from_file("tokenizer/vocab.txt")

In [7]:
tokenized_sentence = tokenizer.encode('Приветствую вас граждане !')
tokenized_sentence.tokens

['[CLS]', 'привет', '##ствую', 'вас', 'граждане', '!', '[SEP]']

In [8]:
# загрузка токенизатора в оболочке transformers
tokenizer = BertTokenizerFast.from_pretrained("tokenizer")

# оболочка LineByLine для более быстрого обучения
dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "corpus.txt",
    block_size = 128
)

# collator MLM для подготовки к обучению
data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = True,
    mlm_probability = 0.15
)

c:\Users\oudel\anaconda3\envs\newenv\lib\site-packages\transformers\data\datasets\language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [9]:
# аргументы обучения 
training_args = TrainingArguments(
    output_dir = "BERT",
    overwrite_output_dir = True,
    num_train_epochs = 1,
    per_device_train_batch_size = 128
)

# создание модели BERT с конфигурацией по умолчанию (т.е. базовая архитектура модели)
tiny_bert_config = BertConfig(
    max_position_embeddings=512,
    hidden_size=128,
    num_attention_heads=2,
    num_hidden_layers=2,
    intermediate_size=512
)

tiny_bert = BertForMaskedLM(tiny_bert_config)


# объект Trainer
trainer = Trainer(
    model = tiny_bert,
    args = training_args,
    data_collator = data_collator,
    train_dataset = dataset
)

# тренировка
trainer.train()

c:\Users\oudel\anaconda3\envs\newenv\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: oudelexsus2010. Use `wandb login --relogin` to force relogin


  0%|          | 0/1029 [00:00<?, ?it/s]

{'loss': 9.1263, 'grad_norm': 1.171692132949829, 'learning_rate': 2.5704567541302237e-05, 'epoch': 0.49}
{'loss': 8.1207, 'grad_norm': 1.0261869430541992, 'learning_rate': 1.4091350826044704e-06, 'epoch': 0.97}
{'train_runtime': 399.7437, 'train_samples_per_second': 329.396, 'train_steps_per_second': 2.574, 'train_loss': 8.605672586068467, 'epoch': 1.0}


TrainOutput(global_step=1029, training_loss=8.605672586068467, metrics={'train_runtime': 399.7437, 'train_samples_per_second': 329.396, 'train_steps_per_second': 2.574, 'train_loss': 8.605672586068467, 'epoch': 1.0})

In [10]:
# сохранение
trainer.save_model("MyBERT")